<a href="https://colab.research.google.com/github/VictorMadu/Data-Science--Cheat-Sheet/blob/master/Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
import os
from scipy import stats
import pickle
import warnings; warnings.filterwarnings('ignore')

In [0]:
df = pd.read_csv("/content/drive/My Drive/fraud detection/data/processed_train.csv")

In [5]:
print(df.shape[0], df.shape[1])

10493 1259


In [6]:
df.head()

,account destination verification,account source verification,account type,age,credit card type,current bank amount,fradulent,id,last bank amount,marital_status,most recent bank amount,occupation,time taken (seconds),transaction source method,is_busday,year,quarter,month,week,day,hour,minute,second,weekday,no_days_in_month,age_group,diff_current_and_last_amount,diff_recent_and_last_amount,diff_recent_and_current_amount,account type_current bank amount_mean,account type_current bank amount_max,account type_current bank amount_min,account type_current bank amount_range,account type_current bank amount_sum,account type_current bank amount_skew,account type_current bank amount_kurtosis,account type_current bank amount_std,account type_current bank amount_count,account type_last bank amount_mean,account type_last bank amount_max,...,no_days_in_month_age_min,no_days_in_month_age_range,no_days_in_month_age_sum,no_days_in_month_age_skew,no_days_in_month_age_kurtosis,no_days_in_month_age_std,no_days_in_month_age_count,no_days_in_month_diff_current_and_last_amount_mean,no_days_in_month_diff_current_and_last_amount_max,no_days_in_month_diff_current_and_last_amount_min,no_days_in_month_diff_current_and_last_amount_range,no_days_in_month_diff_current_and_last_amount_sum,no_days_in_month_diff_current_and_last_amount_skew,no_days_in_month_diff_current_and_last_amount_kurtosis,no_days_in_month_diff_current_and_last_amount_std,no_days_in_month_diff_current_and_last_amount_count,no_days_in_month_diff_recent_and_last_amount_mean,no_days_in_month_diff_recent_and_last_amount_max,no_days_in_month_diff_recent_and_last_amount_min,no_days_in_month_diff_recent_and_last_amount_range,no_days_in_month_diff_recent_and_last_amount_sum,no_days_in_month_diff_recent_and_last_amount_skew,no_days_in_month_diff_recent_and_last_amount_kurtosis,no_days_in_month_diff_recent_and_last_amount_std,no_days_in_month_diff_recent_and_last_amount_count,no_days_in_month_diff_recent_and_current_amount_mean,no_days_in_month_diff_recent_and_current_amount_max,no_days_in_month_diff_recent_and_current_amount_min,no_days_in_month_diff_recent_and_current_amount_range,no_days_in_month_diff_recent_and_current_amount_sum,no_days_in_month_diff_recent_and_current_amount_skew,no_days_in_month_diff_recent_and_current_amount_kurtosis,no_days_in_month_diff_recent_and_current_amount_std,no_days_in_month_diff_recent_and_current_amount_count,current bank amount/time taken (seconds),last bank amount/timetime taken (seconds),most recent bank amount/time taken (seconds),diff_current_and_last_amount/time taken (seconds),diff_recent_and_last_amount/time taken (seconds),diff_recent_and_current_amount/time taken (seconds)
0,1,1,saving,46,master,43671,0,acus0,1151263,married,348764,musician,2299,0,1,2016,4,10,40,5,1,44,58,2,31,4,-1107592,-802499,305093,49890.026256,99917,30,99887,110207068,-0.001333,-1.197369,29083.038329,2209,751193.550023,1499877,...,30,49,473950,0.000565,-1.197589,14.448248,8677,-696990.075141,99489,-1494270,1593759,-6047782882,-0.016360,-1.194548,435851.205717,8677,-498999.923822,483791,-1489198,1972989,-4329822339,-0.007437,-0.987293,457822.152474,8677,197990.151320,497672,-94720,592392,1717960543,0.012826,-1.103505,146751.567276,8677,18.995650,500.766855,151.702479,-481.771205,-349.064376,305093
1,0,1,current,64,master,7442,0,acus1,1062843,single,225114,accounting,4553,0,1,2015,1,1,3,14,15,5,7,2,31,7,-1055401,-837729,217672,50017.612775,99996,1,99995,639775285,-0.004401,-1.193485,28751.805897,12791,750144.725119,1499722,...,30,49,473950,0.000565,-1.197589,14.448248,8677,-696990.075141,99489,-1494270,1593759,-6047782882,-0.016360,-1.194548,435851.205717,8677,-498999.923822,483791,-1489198,1972989,-4329822339,-0.007437,-0.987293,457822.152474,8677,197990.151320,497672,-94720,592392,1717960543,0.012826,-1.103505,146751.567276,8677,1.634527,233.437953,49.443005,-231.803426,-183.994948,217672
2,1,1,current,41,master,40102,0,acus2,48042,married,420486,farmer,1493,1,1,2016,4,10,43,28,13

In [7]:
df[df.isnull().any(1)]

,account destination verification,account source verification,account type,age,credit card type,current bank amount,fradulent,id,last bank amount,marital_status,most recent bank amount,occupation,time taken (seconds),transaction source method,is_busday,year,quarter,month,week,day,hour,minute,second,weekday,no_days_in_month,age_group,diff_current_and_last_amount,diff_recent_and_last_amount,diff_recent_and_current_amount,account type_current bank amount_mean,account type_current bank amount_max,account type_current bank amount_min,account type_current bank amount_range,account type_current bank amount_sum,account type_current bank amount_skew,account type_current bank amount_kurtosis,account type_current bank amount_std,account type_current bank amount_count,account type_last bank amount_mean,account type_last bank amount_max,...,no_days_in_month_age_min,no_days_in_month_age_range,no_days_in_month_age_sum,no_days_in_month_age_skew,no_days_in_month_age_kurtosis,no_days_in_month_age_std,no_days_in_month_age_count,no_days_in_month_diff_current_and_last_amount_mean,no_days_in_month_diff_current_and_last_amount_max,no_days_in_month_diff_current_and_last_amount_min,no_days_in_month_diff_current_and_last_amount_range,no_days_in_month_diff_current_and_last_amount_sum,no_days_in_month_diff_current_and_last_amount_skew,no_days_in_month_diff_current_and_last_amount_kurtosis,no_days_in_month_diff_current_and_last_amount_std,no_days_in_month_diff_current_and_last_amount_count,no_days_in_month_diff_recent_and_last_amount_mean,no_days_in_month_diff_recent_and_last_amount_max,no_days_in_month_diff_recent_and_last_amount_min,no_days_in_month_diff_recent_and_last_amount_range,no_days_in_month_diff_recent_and_last_amount_sum,no_days_in_month_diff_recent_and_last_amount_skew,no_days_in_month_diff_recent_and_last_amount_kurtosis,no_days_in_month_diff_recent_and_last_amount_std,no_days_in_month_diff_recent_and_last_amount_count,no_days_in_month_diff_recent_and_current_amount_mean,no_days_in_month_diff_recent_and_current_amount_max,no_days_in_month_diff_recent_and_current_amount_min,no_days_in_month_diff_recent_and_current_amount_range,no_days_in_month_diff_recent_and_current_amount_sum,no_days_in_month_diff_recent_and_current_amount_skew,no_days_in_month_diff_recent_and_current_amount_kurtosis,no_days_in_month_diff_recent_and_current_amount_std,no_days_in_month_diff_recent_and_current_amount_count,current bank amount/time taken (seconds),last bank amount/timetime taken (seconds),most recent bank amount/time taken (seconds),diff_current_and_last_amount/time taken (seconds),diff_recent_and_last_amount/time taken (seconds),diff_recent_and_current_amount/time taken (seconds)


In [0]:
cat = ['age', 'account type','account source verification', 'transaction source method', 'account destination verification', 'credit card type',  'age_group', 'marital_status', 'occupation', 'is_busday', 'year', 'quarter', 'month', 'week', 'day', 'hour', 'weekday', 'no_days_in_month', 'minute', 'second']
for i in cat:
  df[i] = df[i].astype('category')

In [9]:
df.head()

,account destination verification,account source verification,account type,age,credit card type,current bank amount,fradulent,id,last bank amount,marital_status,most recent bank amount,occupation,time taken (seconds),transaction source method,is_busday,year,quarter,month,week,day,hour,minute,second,weekday,no_days_in_month,age_group,diff_current_and_last_amount,diff_recent_and_last_amount,diff_recent_and_current_amount,account type_current bank amount_mean,account type_current bank amount_max,account type_current bank amount_min,account type_current bank amount_range,account type_current bank amount_sum,account type_current bank amount_skew,account type_current bank amount_kurtosis,account type_current bank amount_std,account type_current bank amount_count,account type_last bank amount_mean,account type_last bank amount_max,...,no_days_in_month_age_min,no_days_in_month_age_range,no_days_in_month_age_sum,no_days_in_month_age_skew,no_days_in_month_age_kurtosis,no_days_in_month_age_std,no_days_in_month_age_count,no_days_in_month_diff_current_and_last_amount_mean,no_days_in_month_diff_current_and_last_amount_max,no_days_in_month_diff_current_and_last_amount_min,no_days_in_month_diff_current_and_last_amount_range,no_days_in_month_diff_current_and_last_amount_sum,no_days_in_month_diff_current_and_last_amount_skew,no_days_in_month_diff_current_and_last_amount_kurtosis,no_days_in_month_diff_current_and_last_amount_std,no_days_in_month_diff_current_and_last_amount_count,no_days_in_month_diff_recent_and_last_amount_mean,no_days_in_month_diff_recent_and_last_amount_max,no_days_in_month_diff_recent_and_last_amount_min,no_days_in_month_diff_recent_and_last_amount_range,no_days_in_month_diff_recent_and_last_amount_sum,no_days_in_month_diff_recent_and_last_amount_skew,no_days_in_month_diff_recent_and_last_amount_kurtosis,no_days_in_month_diff_recent_and_last_amount_std,no_days_in_month_diff_recent_and_last_amount_count,no_days_in_month_diff_recent_and_current_amount_mean,no_days_in_month_diff_recent_and_current_amount_max,no_days_in_month_diff_recent_and_current_amount_min,no_days_in_month_diff_recent_and_current_amount_range,no_days_in_month_diff_recent_and_current_amount_sum,no_days_in_month_diff_recent_and_current_amount_skew,no_days_in_month_diff_recent_and_current_amount_kurtosis,no_days_in_month_diff_recent_and_current_amount_std,no_days_in_month_diff_recent_and_current_amount_count,current bank amount/time taken (seconds),last bank amount/timetime taken (seconds),most recent bank amount/time taken (seconds),diff_current_and_last_amount/time taken (seconds),diff_recent_and_last_amount/time taken (seconds),diff_recent_and_current_amount/time taken (seconds)
0,1,1,saving,46,master,43671,0,acus0,1151263,married,348764,musician,2299,0,1,2016,4,10,40,5,1,44,58,2,31,4,-1107592,-802499,305093,49890.026256,99917,30,99887,110207068,-0.001333,-1.197369,29083.038329,2209,751193.550023,1499877,...,30,49,473950,0.000565,-1.197589,14.448248,8677,-696990.075141,99489,-1494270,1593759,-6047782882,-0.016360,-1.194548,435851.205717,8677,-498999.923822,483791,-1489198,1972989,-4329822339,-0.007437,-0.987293,457822.152474,8677,197990.151320,497672,-94720,592392,1717960543,0.012826,-1.103505,146751.567276,8677,18.995650,500.766855,151.702479,-481.771205,-349.064376,305093
1,0,1,current,64,master,7442,0,acus1,1062843,single,225114,accounting,4553,0,1,2015,1,1,3,14,15,5,7,2,31,7,-1055401,-837729,217672,50017.612775,99996,1,99995,639775285,-0.004401,-1.193485,28751.805897,12791,750144.725119,1499722,...,30,49,473950,0.000565,-1.197589,14.448248,8677,-696990.075141,99489,-1494270,1593759,-6047782882,-0.016360,-1.194548,435851.205717,8677,-498999.923822,483791,-1489198,1972989,-4329822339,-0.007437,-0.987293,457822.152474,8677,197990.151320,497672,-94720,592392,1717960543,0.012826,-1.103505,146751.567276,8677,1.634527,233.437953,49.443005,-231.803426,-183.994948,217672
2,1,1,current,41,master,40102,0,acus2,48042,married,420486,farmer,1493,1,1,2016,4,10,43,28,13

In [0]:
gat =  ['age', 'account type', 'account source verification', 'transaction source method', 'account destination verification', 'credit card type', 'age_group', 'marital_status', 'occupation', 'is_busday', 'year', 'quarter', 'month', 'week', 'day', 'hour', 'weekday', 'no_days_in_month', 'minute', 'second', 'fradulent']
gf = df[gat]
cols_dict = dict()                                 
for col in cat:
  groups = gf.groupby(col)['fradulent'].mean()
  uniques = gf[col].unique()
  uniques_dict = dict(zip(uniques, groups))
  cols_dict[col] = uniques_dict

In [11]:
gf.head()

,age,account type,account source verification,transaction source method,account destination verification,credit card type,age_group,marital_status,occupation,is_busday,year,quarter,month,week,day,hour,weekday,no_days_in_month,minute,second,fradulent
0,46,saving,1,0,1,master,4,married,musician,1,2016,4,10,40,5,1,2,31,44,58,0
1,64,current,1,0,0,master,7,single,accounting,1,2015,1,1,3,14,15,2,31,5,7,0
2,41,current,1,1,1,master,3,married,farmer,1,2016,4,10,43,28,13,4,31,44,58,0
3,66,current,1,1,1,master,8,unknown,clergy,0,2016,4,11,46,20,15,6,30,5,32,0
4,78,current,0,0,1,verve,10,married,accounting,1,2015,3,9,36,1,16,1,30,11,57,1


In [0]:
X = df.drop(['fradulent', 'id'], axis=1)
y = df['fradulent']

In [0]:
from sklearn.base import BaseEstimator, TransformerMixin, clone
class MeanEncoder(BaseEstimator, TransformerMixin):
    
    def __init__(self, cols=[]):
        self.cols = cols 

    def transform(self, X, **transformparams):
        assert isinstance(X, pd.DataFrame) 
        for col in self.cols:
            X[col+" mean_encoded"] = X[col].map(self.mean_mapped_dict[col])
        return X

        
    def fit(self, X, y=None, **fitparams):
      assert isinstance(X, pd.DataFrame)
      global cols_dict 
      self.mean_mapped_dict = cols_dict.copy()
      return self

In [0]:
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV, GridSearchCV
from scipy.stats import uniform, truncnorm, randint

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures, OrdinalEncoder
from sklearn.feature_selection import SelectKBest

cat_feat = df.drop(['fradulent','id'], axis=1).select_dtypes(exclude=['int', 'float']).columns

num_transform1 = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('fs', SelectKBest(k=150)),
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    
])

num_transform2 = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('fs', SelectKBest(k=150))
])


cat_transform1 = Pipeline(steps=[
    # ('mean_encoder', MeanEncoder(cols=cat_feat)),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
    
])

cat_transform2 = Pipeline(steps=[
    # ('mean_encoder', MeanEncoder(cols=cat_feat)),
    ('ordinal', OrdinalEncoder())
    
])


In [35]:
from sklearn.compose import ColumnTransformer
from scipy.stats import uniform, truncnorm, randint
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.linear_model import LogisticRegression



num_feat = X.select_dtypes(include=['int', 'float']).columns
cat_feat = X.select_dtypes(exclude=['int', 'float']).columns

preprocesser = ColumnTransformer(transformers=[
      ('num', num_transform1, num_feat),
      ('cat', cat_transform1, cat_feat)
  ])

clf_pipe = Pipeline(steps=[
    ('preprocesser', preprocesser),
    ('classifier', LogisticRegression(class_weight='balanced', C=0.5))
])


grid_params = { 
            'preprocesser__num__fs__k': randint(80,190),
            'classifier__penalty': ['l1', 'l2', 'elasticnet'],
            'classifier__l1_ratio': uniform(0.00, 1.00),
            'classifier__C': uniform(0.00, 1.00)
}

print('Model training')
clf_search = RandomizedSearchCV(clf_pipe, grid_params, n_iter=100, cv=5, scoring='roc_auc')
clf = clf_search.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print('\n accuracy of train', accuracy_score(y_train, clf.predict(X_train)))
print('\n roc_auc_score of train', roc_auc_score(y_train, clf.predict(X_train)))
print('\naccuarcy score of test', accuracy_score(y_test, y_pred))
print('\nroc auc score of test', roc_auc_score(y_test, y_pred))



# if not os.path.exists("/content/drive/My Drive/fraud detection/models"):
#     os.makedirs("/content/drive/My Drive/fraud detection/models")

# pickle.dump(clf, open(f"/content/drive/My Drive/fraud detection/models/lr", "wb"))
# print("_"*70)
  

Model training

 accuracy of train 0.7221238938053097

 roc_auc_score of train 0.7877819809014163

accuarcy score of test 0.7198221092757306

roc auc score of test 0.7808216528740546


In [0]:
.import pickle
clf = pickle.load(open("/content/drive/My Drive/fraud detection/models/lr", "rb"))

In [18]:
clf.best_params_

{'classifier__C': 0.3988039049238967,
 'classifier__l1_ratio': 0.8434464466957959,
 'classifier__penalty': 'l2',
 'preprocesser__num__fs__k': 167}

In [22]:
clf.best_params_

{'classifier__C': 0.744584134819443,
 'classifier__l1_ratio': 0.9513626131227076,
 'classifier__penalty': 'l2',
 'preprocesser__num__fs__k': 95}

In [30]:
clf.best_params_

{'classifier__C': 0.02579173708282434,
 'classifier__l1_ratio': 0.3969533965417059,
 'classifier__penalty': 'l2',
 'preprocesser__num__fs__k': 176}

In [36]:
clf.best_params_

{'classifier__C': 0.7395678156557826,
 'classifier__l1_ratio': 0.06342549276476517,
 'classifier__penalty': 'l2',
 'preprocesser__num__fs__k': 168}

In [37]:
test_preprocessed = pd.read_csv('/content/drive/My Drive/fraud detection/data/processed_test.csv')
test_preprocessed.head()

,account destination verification,account source verification,account type,age,credit card type,current bank amount,id,last bank amount,marital_status,most recent bank amount,occupation,time taken (seconds),transaction source method,is_busday,year,quarter,month,week,day,hour,minute,second,weekday,no_days_in_month,age_group,diff_current_and_last_amount,diff_recent_and_last_amount,diff_recent_and_current_amount,account type_current bank amount_mean,account type_current bank amount_max,account type_current bank amount_min,account type_current bank amount_range,account type_current bank amount_sum,account type_current bank amount_skew,account type_current bank amount_kurtosis,account type_current bank amount_std,account type_current bank amount_count,account type_last bank amount_mean,account type_last bank amount_max,account type_last bank amount_min,...,no_days_in_month_age_min,no_days_in_month_age_range,no_days_in_month_age_sum,no_days_in_month_age_skew,no_days_in_month_age_kurtosis,no_days_in_month_age_std,no_days_in_month_age_count,no_days_in_month_diff_current_and_last_amount_mean,no_days_in_month_diff_current_and_last_amount_max,no_days_in_month_diff_current_and_last_amount_min,no_days_in_month_diff_current_and_last_amount_range,no_days_in_month_diff_current_and_last_amount_sum,no_days_in_month_diff_current_and_last_amount_skew,no_days_in_month_diff_current_and_last_amount_kurtosis,no_days_in_month_diff_current_and_last_amount_std,no_days_in_month_diff_current_and_last_amount_count,no_days_in_month_diff_recent_and_last_amount_mean,no_days_in_month_diff_recent_and_last_amount_max,no_days_in_month_diff_recent_and_last_amount_min,no_days_in_month_diff_recent_and_last_amount_range,no_days_in_month_diff_recent_and_last_amount_sum,no_days_in_month_diff_recent_and_last_amount_skew,no_days_in_month_diff_recent_and_last_amount_kurtosis,no_days_in_month_diff_recent_and_last_amount_std,no_days_in_month_diff_recent_and_last_amount_count,no_days_in_month_diff_recent_and_current_amount_mean,no_days_in_month_diff_recent_and_current_amount_max,no_days_in_month_diff_recent_and_current_amount_min,no_days_in_month_diff_recent_and_current_amount_range,no_days_in_month_diff_recent_and_current_amount_sum,no_days_in_month_diff_recent_and_current_amount_skew,no_days_in_month_diff_recent_and_current_amount_kurtosis,no_days_in_month_diff_recent_and_current_amount_std,no_days_in_month_diff_recent_and_current_amount_count,current bank amount/time taken (seconds),last bank amount/timetime taken (seconds),most recent bank amount/time taken (seconds),diff_current_and_last_amount/time taken (seconds),diff_recent_and_last_amount/time taken (seconds),diff_recent_and_current_amount/time taken (seconds)
0,0,1,current,78,verve,88677,bcus2993,1059697,unknown,29964,accounting,3033,1,1,2016,2,6,26,28,12,51,43,1,30,10,-971020,-1029733,-58713,50017.612775,99996,1,99995,639775285,-0.004401,-1.193485,28751.805897,12791,750144.725119,1499722,8,...,30,49,277340,-0.016715,-1.188783,14.394111,5056,-703042.104628,97142,-1489412,1586554,-3554580881,0.017912,-1.183684,434988.770042,5056,-502333.386076,481031,-1467547,1948578,-2539797600,0.004821,-1.006667,455073.325606,5056,200708.718552,495223,-96346,591569,1014783281,0.000620,-1.104122,147897.801722,5056,29.237389,349.389054,9.879327,-320.151665,-339.509726,-58713
1,0,1,current,38,master,85737,bcus2994,434908,single,354835,farmer,3025,0,1,2015,4,12,51,16,18,4,3,2,31,2,-349171,-80073,269098,50017.612775,99996,1,99995,639775285,-0.004401,-1.193485,28751.805897,12791,750144.725119,1499722,8,...,30,49,473950,0.000565,-1.197589,14.448248,8677,-696990.075141,99489,-1494270,1593759,-6047782882,-0.016360,-1.194548,435851.205717,8677,-498999.923822,483791,-1489198,1972989,-4329822339,-0.007437,-0.987293,457822.152474,8677,197990.151320,497672,-94720,592392,1717960543,0.012826,-1.103505,146751.567276,8677,28.342810,143.771240,117.300826,-115.428430,-26.470413,269098
2,0,1,current,47,master,14611,bcus2995,1287631,unknown,395445,bank

In [0]:
test_labels = np.round(clf.predict(test_preprocessed)).flatten()

In [39]:
submission = pd.DataFrame({'id':test_preprocessed['id'],'fradulent':test_labels})
submission.head(10)

,id,fradulent
0,bcus2993,0
1,bcus2994,1
2,bcus2995,0
3,bcus2996,1
4,bcus2997,0
5,bcus2998,0
6,bcus2999,0
7,bcus3000,0
8,bcus3001,1
9,bcus3002,1


In [0]:
submission.to_csv('submission_50.csv',index=False)